# Study of Borrower Reliability


## Let's open the table and explore the general information about the data.

**Let's import the pandas library, read the data from the CSV file into a DataFrame, and store it in the variable `data`. File path:**

`/datasets/data.csv`

In [52]:
import pandas as pd

try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Let's display the first 20 rows of the DataFrame `data` on the screen.**

In [28]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Let's display the basic information about the DataFrame using the `info()` method.**

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.6+ MB


## Data preprocessing

### Missing values removal

**Let's display the number of missing values for each column.**

In [30]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**There are missing values in two columns. One of them is `days_employed`. The other column with missing values — `total_income`  — stores income data. Income is most influenced by the type of employment, so we need to fill the missing values in this column using the median income for each employment type from the `income_type`column. For example, if a person has the employment type `сотрудник` their missing `total_income` value should be filled with the median income among all entries with the same type.**

In [31]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Handling anomalous values

**The data may contain artifacts (anomalies) — values that do not reflect reality and appeared due to some error. One such artifact is the negative number of days of work experience in the `days_employed`column. For real-world data, this is common. Let's handle the values in this column by replacing all negative values with positive ones using the `abs()`method.**

In [32]:
data['days_employed'] = data['days_employed'].abs()

**For each employment type, display the median value of work experience `days_employed` in days.**

In [33]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Two types (unemployed and retirees) will have abnormally large values. It's difficult to correct such values, so we'll leave them as they are — especially since this column won't be needed for our analysis.

**Let's display the list of unique values in the `children` column.**

In [34]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**There are two anomalous values in the `children` column. Let's remove the rows containing these anomalous values from the `data` DataFrame.**

In [35]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Let's display the list of unique values in the `children` column once again to make sure the artifacts have been removed.**

In [36]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Missing values removal (continued)

**Let's fill in the missing values in the `days_employed` column with the median values for each employment type from the `income_type` column.**

In [37]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Let’s make sure all missing values have been filled. We’ll double-check by displaying the number of missing values for each column using two methods.**

In [38]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Changing data types

**Let's convert the float data type in the `total_income` column to an integer using the `astype()` method.**

In [39]:
data['total_income'] = data['total_income'].astype(int)

### Handling duplicates

**Let’s handle implicit duplicates in the `education` column. This column contains the same values written differently — using uppercase and lowercase letters. We'll convert them all to lowercase. Then, we’ll check the other columns for similar inconsistencies.**

In [40]:
data['education'] = data['education'].str.lower()

**Let's display the number of duplicate rows in the data. If such rows are present, we’ll remove them.**

In [41]:
data.duplicated().sum()

np.int64(71)

In [42]:
data = data.drop_duplicates()

### Data categorization

**Based on the ranges listed below, let's create a new column in the `data` DataFrame called `total_income_category` with the following categories:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**For example, a borrower with an income of 25,000 should be assigned category `'E'`, and a client earning 235,000 should get category `'B'`. We'll use a custom function named `categorize_income()` and apply it using the `apply()` method.**

In [43]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [44]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Let's display the list of unique loan purposes from the  `purpose` column.**

In [45]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Let's create a function that generates a new column `purpose_category` based on the values in the `purpose`column. The categories will be:**

- `'операции с автомобилем'` (car-related operations),
- `'операции с недвижимостью'` (real estate operations),
- `'проведение свадьбы'` (wedding),
- `'получение образования'` (education).

**For example, if the string `'на покупку автомобиля'` is found in the `purpose` column, the corresponding value in `purpose_category` should be `'операции с автомобилем'`.

We’ll use a custom function named `categorize_purpose()` along with the `.apply()` method. We'll also analyze the values in `purpose` to determine which substrings can help us correctly assign each category.**

In [46]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [47]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Let's explore the data and answer the questions.

#### Is there a correlation between the number of children and timely loan repayment?

In [48]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
children_pivot = data.pivot_table(index='children', values='debt', aggfunc=['count', 'mean'])

children_pivot.columns = ['Number of clients', 'Share of debtors']

children_pivot['Clients (%)'] = (children_pivot['Number of clients'] / data.shape[0] * 100).round(2)
children_pivot['Debtors (%)'] = (children_pivot['Share of debtors'] * 100).round(2)

children_pivot = children_pivot.reset_index()
children_pivot

,children,Number of clients,Share of debtors,Clients (%),Debtors (%)
0,0,14091,0.075438,66.06,7.54
1,1,4808,0.092346,22.54,9.23
2,2,2052,0.094542,9.62,9.45
3,3,330,0.081818,1.55,8.18
4,4,41,0.097561,0.19,9.76
5,5,9,0.000000,0.04,0.00


**Conclusion:** Clients without children have a share of debtors of 7.5%. For those with 1–2 children, it’s around 9.3–9.5%. Groups with 3 or more children are less representative, but overdue payments also occur. The more children a client has, the higher the risk of default — especially noticeable for those with 1–2 children.

#### Is there a correlation between marital status and timely loan repayment?

In [49]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
family_pivot = data.pivot_table(index='family_status', values='debt', aggfunc=['count', 'mean'])

family_pivot.columns = ['Number of clients', 'Share of debtors']

family_pivot['Clients (%)'] = (family_pivot['Number of clients'] / data.shape[0] * 100).round(2)
family_pivot['Debtors (%)'] = (family_pivot['Share of debtors'] * 100).round(2)

family_pivot = family_pivot.reset_index()

family_pivot


,family_status,Number of clients,Share of debtors,Clients (%),Debtors (%)
0,Не женат / не замужем,2796,0.097639,13.11,9.76
1,в разводе,1189,0.070648,5.57,7.06
2,вдовец / вдова,951,0.066246,4.46,6.62
3,гражданский брак,4134,0.093130,19.38,9.31
4,женат / замужем,12261,0.075606,57.48,7.56


**Conclusion:** Married clients, as well as those who are divorced or widowed, are less likely to have outstanding debts compared to single clients and those in a civil partnership. There is a correlation between marital status and timely loan repayment.

#### Is there a correlation between income level and timely loan repayment?

In [50]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
income_pivot = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'mean'])

income_pivot.columns = ['Number of clients', 'Share of debtors']

income_pivot['Clients (%)'] = (income_pivot['Number of clients'] / data.shape[0] * 100).round(2)
income_pivot['Debtors (%)'] = (income_pivot['Share of debtors'] * 100).round(2)

income_pivot = income_pivot.reset_index()

income_pivot


,total_income_category,Number of clients,Share of debtors,Clients (%),Debtors (%)
0,A,25,0.080000,0.12,8.00
1,B,5014,0.070602,23.51,7.06
2,C,15921,0.084982,74.64,8.50
3,D,349,0.060172,1.64,6.02
4,E,22,0.090909,0.10,9.09


**Conclusion:** Clients with income category D have the lowest share of debtors, while those in category E have the highest. However, the differences in proportions are minor, so there is no clear correlation between income level and timely loan repayment.

#### How do different loan purposes affect timely repayment?

In [51]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
purpose_pivot = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'mean'])

purpose_pivot.columns = ['Number of clients', 'Share of debtors']

purpose_pivot['Clients (%)'] = (purpose_pivot['Number of clients'] / data.shape[0] * 100).round(2)
purpose_pivot['Debtors (%)'] = (purpose_pivot['Share of debtors'] * 100).round(2)

purpose_pivot = purpose_pivot.reset_index()

purpose_pivot


,purpose_category,Number of clients,Share of debtors,Clients (%),Debtors (%)
0,операции с автомобилем,4279,0.093480,20.06,9.35
1,операции с недвижимостью,10751,0.072551,50.40,7.26
2,получение образования,3988,0.092528,18.70,9.25
3,проведение свадьбы,2313,0.079118,10.84,7.91


**Conclusion:** The lowest share of debtors is among those who took loans for real estate. The highest share is observed for loans taken for cars and education. This suggests that the purpose of the loan may influence the likelihood of default.

#### Let's list possible reasons for missing values in the original data.

*Answer:*  
        Human factor — the client might not have provided the information when submitting the application.
        Technical errors — failures during data collection or transfer from external sources.
        Field inapplicability — for example, lack of work experience for students or unemployed individuals.
        Outdated or incomplete databases — the information might have been outdated or not updated at the time of export.


### Overall conclusion.

Based on the conducted analysis, the following conclusions can be made:

- **Number of children** affects loan repayment. Clients **without children** have a debtor share of **7.5%**, while those with **one or two children** have **around 9.2–9.5%**. For clients with **three or more children**, the sample is small, but the debtor share is also high.

- **Marital status** also matters. Among **married** and **divorced** clients, the debtor share is **below 8%**, whereas among **single** and those in a **civil partnership**, it is **about 9.7%**.

- **Income level** has a weak influence, but still noticeable: category **D** (income from 30k to 50k) shows the **lowest debtor share (6.9%)**, while category **E** (up to 30k) has **the highest (10.3%)**. Other categories have similar values ranging from 7.3% to 8.5%.

- **Loan purpose** significantly influences repayment. The lowest debtor share is among clients who took loans for **real estate (7.3%)**, and the highest is for loans for **cars (9.3%)** and **education (9.2%)**.

Thus, timely loan repayment is most likely among clients **without children, married, with medium income, and whose loan purpose is real estate**.